In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# spacy for lemmatization
import spacy
import json
import warnings
import networkx as nx

warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk
nltk.download('stopwords')
nltk.download('words')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.corpus import words
eng_words = words.words('en')

from langdetect import detect
# to enforce consistent results, check github langdetect readme
from langdetect import DetectorFactory
DetectorFactory.seed = 0


import os 

import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheesan_manokaran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/matheesan_manokaran/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
df = pd.read_csv('mount/TweetsOnUser.csv', lineterminator='\n', low_memory=False)
df.head()

,id,TweetText,userid,creationtimestamp
0,7515433917751296,Enjoy TSA Pat Down RT Hey everybody Thanksgivi...,194306325,2010-11-24 19:26:33 UTC
1,18960074751868928,Via RT Breaking News Assange signs book deal W...,86711072,2010-12-26 09:23:28 UTC
2,17700051287547904,state of heart soul your soul Birthing child m...,39126647,2010-12-22 21:56:35 UTC
3,21446749583314945,Sweet dreams,23262180,2011-01-02 06:04:38 UTC
4,18528789873041408,dramatic real Operation Clean-Up Now RT CRAP R...,71496690,2010-12-25 04:49:42 UTC


In [3]:
tweets = df.TweetText.values.tolist()
userIds = df.userid.values.tolist()
print("PLS")

PLS


In [4]:
dict = {}

for i in range(len(userIds)):
    if userIds[i] not in dict:
        dict[userIds[i]] = []
    for word in str(tweets[i]).split(" "):
        dict[userIds[i]].append(word)


In [5]:
data_final = list(dict.values())

In [6]:
# Create Dictionary
id2word = corpora.Dictionary(data_final)

# Create Corpus
# type(data_lemmatized)
# print (data_lemmatized)
texts = data_final

print(data_final[:1])

[['you!', 'I', 'love', 'xoxo', 'you!', 'I', 'proud', 'of', 'you', 'I', 'love', 'you', 'my', 'day', 'day.', 'After', 'After', 'all', 'all,', 'I', 'was', 'I', 'was', 'born', 'spread', 'I', 'am', 'love', "I'm", 'sadness', 'today', 'who', 'I', 'am', "I'm", 'thankful', 'everything', 'everyone', 'good', 'and', 'the', 'bad', 'happiness']]


In [7]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Human readable format of corpus (term-frequency)
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])

# Build LDA model
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=100, passes=10, workers=7)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

[[('After', 2), ('I', 7), ("I'm", 2), ('all', 1), ('all,', 1), ('am', 2), ('and', 1), ('bad', 1), ('born', 1), ('day', 1), ('day.', 1), ('everyone', 1), ('everything', 1), ('good', 1), ('happiness', 1), ('love', 3), ('my', 1), ('of', 1), ('proud', 1), ('sadness', 1), ('spread', 1), ('thankful', 1), ('the', 1), ('today', 1), ('was', 2), ('who', 1), ('xoxo', 1), ('you', 2), ('you!', 2)]]
[(55,
  '0.098*"Santa" + 0.077*"San" + 0.035*"Francisco" + 0.023*"Claus" + '
  '0.019*"SF" + 0.017*"Ho" + 0.015*"Diego" + 0.014*"Dear" + 0.010*"Bay" + '
  '0.008*"NORAD"'),
 (31,
  '0.014*"Vegas" + 0.010*"Las" + 0.009*"today" + 0.007*"Airport" + 0.007*"mph" '
  '+ 0.006*"Rain" + 0.006*"mb" + 0.006*"is" + 0.006*"Wind" + 0.006*"closed"'),
 (19,
  '0.043*"RT" + 0.028*"la" + 0.018*"des" + 0.018*"les" + 0.018*"et" + '
  '0.016*"pour" + 0.016*"du" + 0.016*"en" + 0.015*"le" + 0.015*"un"'),
 (96,
  '0.042*"CA" + 0.030*"Jobs" + 0.029*"Job" + 0.028*"DAILY" + 0.023*"shark" + '
  '0.015*"Los" + 0.014*"Manager" + 0.0

In [8]:
# def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized):
#     # Init output
#     sent_topics_df = pd.DataFrame()

#     # Get main topic in each document
#     for i, row in enumerate(ldamodel[corpus]):
#         print(row) 
#         topics = []
#         row = sorted(row, key=lambda x: (x[1]), reverse=True) 
#         for j, (topic_num, prop_topic) in enumerate(row):
#             if j in [0,1,2]: 
#                 topics.append(topic_num)
#         # Get the Dominant topic, Perc Contribution and Keywords for each document
#         for j, (topic_num, prop_topic) in enumerate(row):
# #             if j in [0,1,2]: #top 3 dominant topics
#             if j == 0:  # => dominant topic
#                 wp = ldamodel.show_topic(topic_num)
#                 topic_keywords = ", ".join([word for word, prop in wp])
#                 sent_topics_df = sent_topics_df.append(pd.Series([topics, round(prop_topic,4), topic_keywords]), ignore_index=True)
#             else:
#                 break
#     sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_final):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_final)

In [9]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
#TER
# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,25.0,0.6109,"RT, you, I, me, I'm, , my, lol, can, up","[you!, I, love, xoxo, you!, I, proud, of, you,..."
1,1,4.0,0.7433,"you, RT, can, life, will, the, who, people, lo...","[H., Jackson, Brown,, Jr, screwdriver, RT, Inf..."
2,2,85.0,0.6729,"RT, Obama, you, will, GOP, tax, we, who, the, ...","[opinion, Sen, Cornyn, Repeal, Don’t, Ask,Don’..."
3,3,4.0,0.4018,"you, RT, can, life, will, the, who, people, lo...","[Hope, stepson, hear, people, power, power, is..."
4,4,50.0,0.2841,"RT, game, , NFL, will, out, up, win, the, season","[Follow, Official, Goon, Gang, Djs, Week, 1, T..."
5,5,63.0,0.4881,"RT, , you, out, will, the, now, of, today, we","[charity:water, Paul, 1:1, 1, time, with, me, ..."
6,6,25.0,0.4790,"RT, you, I, me, I'm, , my, lol, can, up","[Blue, Laws, Christmas, outlawed, Boston, you,..."
7,7,16.0,0.6171,"you, Christmas, Happy, New, Thanksgiving, Year...","[Pink, Floyd, Wall, concert, last, night, pret..."
8,8,58.0,0.5559,", RT, health, you, Health, food, cancer, to, h...","[Search, the, best, doctor, near, you, fb, wei..."
9,9,59.0,0.4939,"E, T, card, O, Panamanian, A, R, L, S, BANK","[Prototype, lanzó, su, 2011, e, Ivan, de, Pine..."


In [ ]:
df_dominant_topic.to_csv('testForDomTopicMM.csv')